Quick analysis to check new data

In [ ]:
#!/usr/bin/env python3
import os
import json
import h5py
import argparse
import pandas as pd
import numpy as np
import tinydb as db
from tinydb.storages import MemoryStorage
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.colors import LogNorm

import boost_histogram as bh
import pickle as pl

import scipy.stats as stats
import scipy.optimize as opt
from pygama import DataGroup
import pygama.lh5 as lh5
import pygama.analysis.histograms as pgh
import pygama.analysis.peak_fitting as pgf
import cage_utils
plt.rcParams['figure.figsize'] = [14, 10]

In [ ]:
run = 60 #44 #70 #64
dsp_id = '02'
user = True
hit =True
cal = True
lowE=False
etype = 'trapEftp_cal'

corr_DCR=True
corr_AoE=True
corr_ToE=True

norm = True
cut = True


dsp_list = ['energy', 'trapEftp', 'trapEmax', 'trapEftp_cal', 'bl','bl_sig', 'bl_slope', 'AoE', 'dcr', "tp_0", "tp_02", "tp_05", "tp_10", "tp_20", "tp_30", "tp_40", "tp_50", "tp_60", "tp_70", "tp_80", "tp_90", "tp_96", 'tp_max', 'ToE', 'log_tail_fit_slope', 'wf_max', 'wf_argmax', 'trapE_argmax', 'lf_max']

df_raw, dg, runtype, rt_min, radius, angle_det, rotary = cage_utils.getDataFrame(run, user=user, hit=hit, cal=cal, dsp_list=dsp_list, lowE=lowE)

df = cage_utils.apply_DC_Cuts(run, df_raw)



In [ ]:
# select energy type

#trapEmax (uncal)
# etype = 'trapEmax'
# elo, ehi, epb = 0, 3000, 1
# blo, bhi, bpb = 0, 10000, 100

#trapEmax_cal (cal)
# etype = 'trapEmax_cal'
# elo, ehi, epb = 0, 5000, 10

# onboard energy
# etype = 'energy'
# elo, ehi, epb = 0, 1000000, 100

# # trapEftp (uncal)
# etype = 'trapEftp'
# elo, ehi, epb = 0, 10000, 10

# trapEftp_cal (cal)
etype = 'trapEftp_cal'
elo, ehi, epb = 0, 6000, 10




In [ ]:
# Baseline vs E
# %matplotlib widget

blo, bhi, bpb = 9500, 11000, 1
nbx = int((ehi-elo)/epb)
nby = int((bhi-blo)/bpb)

h = plt.hist2d(df[etype], df['bl'], bins=[nbx,nby],
                       range=[[elo, ehi], [blo, bhi]], cmap='jet')

cb = plt.colorbar(h[3], ax=plt.gca())
plt.xlabel(f'{etype}', ha='right', x=1)
plt.ylabel('bl', ha='right', y=1)
plt.title('Baseline, alpha')
# plt.tight_layout()
plt.show()

In [ ]:
blo, bhi, bpb = 8800, 9800, 1
nbx = int((bhi-blo)/bpb)

bl_hist, bins = np.histogram(df['bl'], bins=nbx,
                range=[blo, bhi])

plt.semilogy(bins[1:], bl_hist, ds='steps', c='b', lw=1)

bl_cut_lo, bl_cut_hi = 9150,9320

plt.axvline(bl_cut_lo, c='r', lw=1)
plt.axvline(bl_cut_hi, c='r', lw=1)

plt.xlabel('bl', ha='right', x=1)
plt.ylabel('counts', ha='right', y=1)
plt.show()

In [ ]:
# Baseline vs E

bslo, bshi, bspb = -5, 10, 0.1
nbx = int((ehi-elo)/epb)
nby = int((bshi-bslo)/bspb)

h = plt.hist2d(df[etype], df['bl_slope'], bins=[nbx,nby],
                       range=[[elo, ehi], [bslo, bshi]], cmap='jet', norm=LogNorm())

cb = plt.colorbar(h[3], ax=plt.gca())
plt.xlabel(f'{etype}', ha='right', x=1)
plt.ylabel('bl_slope', ha='right', y=1)
plt.title('Baseline slope')
# plt.tight_layout()
plt.show()

In [ ]:
bslo, bshi, bspb = -5, 10, 0.1
nbx = int((bshi-bslo)/bspb)

bl_slope_hist, bins = np.histogram(df['bl_slope'], bins=nbx,
                range=[bslo, bshi])

plt.semilogy(bins[1:], bl_slope_hist, ds='steps', c='b', lw=1)

bl_slo_cut_lo, bl_slo_cut_hi = -4,4

plt.axvline(bl_slo_cut_lo, c='r', lw=1)
plt.axvline(bl_slo_cut_hi, c='r', lw=1)

plt.xlabel('bl_slope', ha='right', x=1)
plt.ylabel('counts', ha='right', y=1)
plt.show()

In [ ]:
# Baseline vs E

bsiglo, bsighi, bsigpb = 2, 7, 0.01
nbx = int((ehi-elo)/epb)
nby = int((bsighi-bsiglo)/bsigpb)

h = plt.hist2d(df[etype], df['bl_sig'], bins=[nbx,nby],
                       range=[[elo, ehi], [bsiglo, bsighi]], cmap='jet', norm=LogNorm())

cb = plt.colorbar(h[3], ax=plt.gca())
plt.xlabel(f'{etype}', ha='right', x=1)
plt.ylabel('bl_sig', ha='right', y=1)
plt.title('Baseline Sig, alpha')
# plt.tight_layout()
plt.show()

In [ ]:
# Baseline vs E
elo, ehi, epb = 0, 6000, 10
lf_lo, lf_hi, lf_pb = 0, 1000000, 1000
nbx = int((ehi-elo)/epb)
nby = int((lf_hi-lf_lo)/lf_pb)

h = plt.hist2d(df[etype], df['lf_max'], bins=[nbx,nby],
                       range=[[elo, ehi], [lf_lo, lf_hi]], cmap='jet', norm=LogNorm())

cb = plt.colorbar(h[3], ax=plt.gca())
plt.xlabel(f'{etype}', ha='right', x=1)
plt.ylabel('lf_max', ha='right', y=1)
plt.title('Low Freq, alpha')
# plt.tight_layout()
plt.show()

In [ ]:
# # Baseline vs E
# elo, ehi, epb = 0, 10000, 10
# lf_lo, lf_hi, lf_pb = 0, 5000000, 1000
# nbx = int((ehi-elo)/epb)
# nby = int((lf_hi-lf_lo)/lf_pb)

# h = plt.hist2d(df['ts_s'], df['lf_max'], bins=[nbx,nby],
#                        range=[[elo, ehi], [lf_lo, lf_hi]], cmap='jet', norm=LogNorm())

# cb = plt.colorbar(h[3], ax=plt.gca())
# plt.xlabel(f'{etype}', ha='right', x=1)
# plt.ylabel('lf_max', ha='right', y=1)
# plt.title('Low Freq, alpha')
# # plt.tight_layout()
# plt.show()

In [ ]:
# use baseline cut
df_cut = df.query(f'bl > {bl_cut_lo} and bl < {bl_cut_hi}').copy()


#create dcr/E 
# df_cut['dcr_norm'] = df_cut['dcr'] / df_cut['trapEftp']

#create A/E cal 
# df_cut['AoE_cal'] = df_cut['A_10'] / df_cut['trapEftp_cal']

#creat new DCR
# const = 0.0555
# df_cut['dcr_linoff'] = df_cut['dcr_raw'] + const*df_cut['trapEmax']

# #create 10-90
# df_cut['10-90'] = df_cut['tp_90']- df_cut['tp_10']

# #create 50-90
# df_cut['50-90'] = df_cut['tp_90']- df_cut['tp_50']

#create 0-50
# df_cut['tp0_50'] = df_cut['tp_50']- df_cut['tp_0']

In [ ]:
# %matplotlib widget
etype = 'trapEftp_cal'
# Make energy spectrum
# elo, ehi, epb = 2775, 2840, 1 #1460 pk uncal trapEmax
# elo, ehi, epb = 0, 5000000, 100 # basically entire range, rawe parameter ("energy")
# elo, ehi, epb = 1440000, 1470000, 100 # 1460 pk, rawe parameter ("energy")
# elo, ehi, epb = 2680000, 2720000, 10 # 2615 peak, rawe parameter ("energy")
# elo, ehi, epb = 2000000, 2500000, 100 # where alphas are in run 117, rawe parameter ("energy")
# elo, ehi, epb = 0, 10000, 10 #trapEftp
# elo, ehi, epb = 100, 20000, 10 #trapEftp uncal


# elo, ehi, epb = 0, 3000, 10
elo, ehi, epb = 0, 6000, 1 #trapEftp_cal
        

nbx = int((ehi-elo)/epb)

energy_hist, bins = np.histogram(df_cut[etype], bins=nbx,
                range=[elo, ehi])
energy_rt = np.divide(energy_hist, rt_min * 60)

plt.semilogy(bins[1:], energy_rt, ds='steps', c='b', lw=1) #, label=f'{etype}'

plt.xlabel(etype, ha='right', x=1)
plt.ylabel('cts / sec', ha='right', y=1)
# plt.legend()
plt.title(f'{etype}: r = {radius} mm; theta = {angle_det} deg\n{runtype} run {run}, {rt_min:.2f} mins')
# plt.savefig(f'./plots/normScan/{runtype}_energy_run{run}.pdf', dpi=200)
# plt.xlim(20,80)
plt.show()

In [ ]:
# # Make calibrated energy spectrum
# elo, ehi, epb = 0, 8000, 10
# # elo, ehi, epb = 0, 3000, 10
# # elo, ehi, epb = 0, 6000, 10
        

# nbx = int((ehi-elo)/epb)

# energy_hist, bins = np.histogram(df_cut['trapEmax_cal'], bins=nbx,
#                 range=[elo, ehi])
# energy_rt = np.divide(energy_hist, rt_min * 60)

# plt.semilogy(bins[1:], energy_rt, ds='steps', c='b', lw=1) #, label=f'{etype}'

# plt.xlabel(etype, ha='right', x=1)
# plt.ylabel('cts / sec', ha='right', y=1)
# # plt.legend()
# plt.title(f'{etype}: r = {radius} mm; theta = {angle_det} deg\n{runtype} run {run}, {rt_min:.2f} mins')
# plt.savefig(f'./plots/normScan/{runtype}_energy_run{run}.pdf', dpi=200)
# plt.show()

In [ ]:
# Alpha A/E
alo, ahi, apb = 0.0, 0.125, 0.001
# elo, ehi, epb = 0, 10000, 10
# elo, ehi, epb = 0, 3000, 10
elo, ehi, epb = 0, 6000, 10
        

nbx = int((ehi-elo)/epb)
nby = int((ahi-alo)/apb)

h = plt.hist2d(df_cut[etype], df_cut['AoE'], bins=[nbx,nby],
                range=[[elo, ehi], [alo, ahi]], cmap='jet', norm=LogNorm())

plt.xlabel(etype, ha='right', x=1)
plt.ylabel('A/E', ha='right', y=1)
# plt.legend()
# plt.title(f'A/E: r = {radius} mm; theta = {angle_det} deg\n{runtype} run {run}, {rt_min:.2f} mins')
plt.tight_layout()
# plt.savefig(f'./plots/normScan/{runtype}_a10oE_run{run}.png', dpi=200)
plt.show()

In [ ]:
# DCR vs E
# etype = 'trapEmax'
# elo, ehi, epb = 500, 1500, 10
elo, ehi, epb = 0, 6000, 10
# elo, ehi, epb = 0, 20000, 10
# dlo, dhi, dpb = -0.08, 0.025, 0.0005
# dlo, dhi, dpb = -10., 30, 0.1
dlo, dhi, dpb = -20., 50, 0.1 #dsp_id= 02
nbx = int((ehi-elo)/epb)
nby = int((dhi-dlo)/dpb)

alpha_dcr_hist = plt.hist2d(df_cut[etype], df_cut['dcr'], bins=[nbx,nby],
                range=[[elo, ehi], [dlo, dhi]], cmap='jet', norm=LogNorm())

plt.xlabel(etype, ha='right', x=1)
plt.ylabel('DCR', ha='right', y=1)
plt.title(f'DCR: {runtype} run {run}, {rt_min:.2f} mins')
plt.tight_layout()
# plt.savefig(f'./plots/normScan/{runtype}_dcr_norm_run{run}.png', dpi=200)
plt.show()

In [ ]:
# AoE vs E
# etype = 'trapEmax'
# elo, ehi, epb = 500, 1500, 10
elo, ehi, epb = 0, 6000, 10
# elo, ehi, epb = 0, 20000, 10
# dlo, dhi, dpb = -0.08, 0.025, 0.0005
# dlo, dhi, dpb = -10., 30, 0.1
alo, ahi, apb= 0.005, 0.075, 0.0001
nbx = int((ehi-elo)/epb)
nby = int((ahi-alo)/apb)

alpha_dcr_hist = plt.hist2d(df_cut[etype], df_cut['AoE'], bins=[nbx,nby],
                range=[[elo, ehi], [alo, ahi]], cmap='jet', norm=LogNorm())

plt.xlabel(etype, ha='right', x=1)
plt.ylabel('A/E', ha='right', y=1)
plt.title(f'A/e: {runtype} run {run}, {rt_min:.2f} mins')
plt.tight_layout()
# plt.savefig(f'./plots/normScan/{runtype}_dcr_norm_run{run}.png', dpi=200)
plt.show()